In [1]:
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
dfJobs = pd.read_csv('VectorMovingJobs.csv')

In [3]:
#!pip install gmplot   # Google Maps plotter
import gmplot
import webbrowser

In [ ]:
def getMonth(dfJobs, mon=1):
    
    

In [4]:
def mapJobs(dfJobs, fileName = 'vectorMovingJobs.html'):

    gmap = gmplot.GoogleMapPlotter(32.7157, -117.1611, 9)  # Centered on San Diego, CA


    #gmap.plot(lats, longs, 'cornflowerblue', edge_width=10) # I think this plots line in between
                                                             # So good for paths

    # This code just plots circles at the lat/lon coordinates
    gmap.scatter(dfJobs[~dfJobs['ORIGIN LATITUDE'].isnull()]['ORIGIN LATITUDE'], 
                 dfJobs[~dfJobs['ORIGIN LATITUDE'].isnull()]['ORIGIN LONGITUDE'], 
                 color = 'm', size = 70, marker=False, alpha=0.5)

    gmap.scatter(dfJobs[~dfJobs['DESTINATION LATITUDE'].isnull()]['DESTINATION LATITUDE'], 
                 dfJobs[~dfJobs['DESTINATION LATITUDE'].isnull()]['DESTINATION LONGITUDE'], 
                 color = 'teal', size = 70, marker=False, alpha=0.5)

    grad1 = [  # Blue to red gradient
        [0, 255, 255, 0],
        [0, 255, 255, 1],
        [0, 191, 255, 1],
        [0, 127, 255, 1],
        [0, 63, 255, 1],
        [0, 0, 255, 1],
        [0, 0, 223, 1],
        [0, 0, 191, 1],
        [0, 0, 159, 1],
        [0, 0, 127, 1],
        [63, 0, 91, 1],
        [127, 0, 63, 1],
        [191, 0, 31, 1],
        [255, 0, 0, 1]]

    # This code adds heatmaps to the lat/lon coordinates
    gmap.heatmap(dfJobs[~dfJobs['ORIGIN LATITUDE'].isnull()]['ORIGIN LATITUDE'], 
                 dfJobs[~dfJobs['ORIGIN LATITUDE'].isnull()]['ORIGIN LONGITUDE'], 
                 radius=10, threshold=10, 
                 gradient=grad1)

    gmap.heatmap(dfJobs[~dfJobs['DESTINATION LATITUDE'].isnull()]['DESTINATION LATITUDE'], 
                 dfJobs[~dfJobs['DESTINATION LATITUDE'].isnull()]['DESTINATION LONGITUDE'], 
                 radius=10, threshold=10)

    gmap.draw(fileName)
    
    webbrowser.open_new_tab(fileName)
    

In [6]:
import folium

In [7]:
from folium.plugins import MarkerCluster

In [116]:
m = folium.Map(location=[32.7157, -117.1611],
                  #tiles='Cartodb Positron', 
               zoom_start=7, control_scale=True)

idsO = ~dfJobs['ORIGIN LATITUDE'].isnull()  # Get all non-null coordinates

locationsO = list(zip(dfJobs[idsO]['ORIGIN LATITUDE'], 
                 dfJobs[idsO]['ORIGIN LONGITUDE']))

dfTemp = dfJobs[idsO].reset_index()
labelsO = []*dfTemp.shape[0]

for i in range(dfTemp.shape[0]):
#     html = folium.Html('{}<br><a href="tel:{}"></a>{}<br><a href="mailto:{}">{}</a>'.format(
#         dfTemp['CUSTOMER NAME'].ix[i],
#         dfTemp['CUSTOMER PHONE'].ix[i],
#         dfTemp['CUSTOMER PHONE'].ix[i],
#         dfTemp['CUSTOMER EMAIL'].ix[i],
#         dfTemp['CUSTOMER EMAIL'].ix[i]), script=True)
    #labelsO.append(folium.Popup(html, max_width=200))
    labelsO.append('ORIGIN FOR: {}'.format(dfTemp['CUSTOMER NAME'].ix[i]))
    
    
idsD = ~dfJobs['DESTINATION LATITUDE'].isnull()  # Get all non-null coordinates

locationsD = list(zip(dfJobs[idsD]['DESTINATION LATITUDE'], 
                 dfJobs[idsD]['DESTINATION LONGITUDE']))


dfTemp = dfJobs[idsD].reset_index()
labelsD = []*dfTemp.shape[0]

for i in range(dfTemp.shape[0]):
#     html = folium.Html('{}<br><a href="tel:{}"></a>{}<br><a href="mailto:{}">{}</a>'.format(
#         dfTemp['CUSTOMER NAME'].ix[i],
#         dfTemp['CUSTOMER PHONE'].ix[i],
#         dfTemp['CUSTOMER PHONE'].ix[i],
#         dfTemp['CUSTOMER EMAIL'].ix[i],
#         dfTemp['CUSTOMER EMAIL'].ix[i]), script=True)
    #labelsD.append(folium.Popup(html, max_width=200))
    labelsD.append('DESTINATION FOR: {}'.format(dfTemp['CUSTOMER NAME'].ix[i]))
  
o = folium.FeatureGroup(name='Origin')
o.add_child(MarkerCluster(locations=locationsO, popups=labelsO))


d = folium.FeatureGroup(name='Destination')
d.add_child(MarkerCluster(locations=locationsD, popups=labelsD))

m.add_child(o)
m.add_child(d)
m.add_child(folium.LayerControl())

m.save('jobsFolium.html')   # Save as html file

m  # Display in notebook


In [115]:
# Build the basic map
jobs_heatmap = folium.Map(location=[32.7157, -117.1611],
                  #tiles='Cartodb Positron', 
                          zoom_start=7, control_scale=True)

idsO = ~dfJobs['ORIGIN LATITUDE'].isnull()  # Get all non-null coordinates

locationsO = list(zip(dfJobs[idsO]['ORIGIN LATITUDE'], 
                 dfJobs[idsO]['ORIGIN LONGITUDE']))

idsD = ~dfJobs['DESTINATION LATITUDE'].isnull()  # Get all non-null coordinates

locationsD = list(zip(dfJobs[idsD]['DESTINATION LATITUDE'], 
                 dfJobs[idsD]['DESTINATION LONGITUDE']))

o = folium.FeatureGroup(name='Origin')
o.add_child(folium.plugins.HeatMap(locationsO, gradient={0.1: 'blue', 0.3: 'lime', 0.9: 'red'}))

d = folium.FeatureGroup(name='Destination')
d.add_child(folium.plugins.HeatMap(locationsD, gradient={0.1: 'red', 0.3: 'lime', 0.9: 'blue'}))

jobs_heatmap.add_child(o)
jobs_heatmap.add_child(d)
jobs_heatmap.add_child(folium.LayerControl())

# Save the map to the local directory
jobs_heatmap.save("./jobs_heatmap.html")

# View the map in Jupyter notebook
jobs_heatmap

In [109]:
def heatMapByMonth(dfJobsOriginal, jobs_heatmap, locationType='ORIGIN'):

    mon = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    
    for m in range(12):
        
        dfJobs = dfJobsOriginal[pd.DatetimeIndex(dfJobsOriginal['DTSTART']).month == (m+1)]
        
        ids = ~dfJobs[locationType+' LATITUDE'].isnull()  # Get all non-null coordinates

        locations = list(zip(dfJobs[ids][locationType+' LATITUDE'], 
                         dfJobs[ids][locationType+' LONGITUDE']))

        o = folium.FeatureGroup(name=mon[m])

        o.add_child(folium.plugins.HeatMap(locations))

        jobs_heatmap.add_child(o)
    
    jobs_heatmap.add_child(folium.LayerControl())

    # Save the map to the local directory
    #jobs_heatmap.save("./jobs_heatmap.html")

    # View the map in Jupyter notebook
    #jobs_heatmap
    
    return jobs_heatmap

In [112]:
# Build the basic map
jobs_heatmap = folium.Map(location=[32.7157, -117.1611], zoom_start=7, control_scale=True)

jobsmap = heatMapByMonth(dfJobs, jobs_heatmap, 'ORIGIN')

jobsmap

jobs_heatmap.save("./jobs_heatmap_by_month_origin.html")
    

In [113]:
# Build the basic map
jobs_heatmap = folium.Map(location=[32.7157, -117.1611], zoom_start=7, control_scale=True)

jobsmap = heatMapByMonth(dfJobs, jobs_heatmap, 'DESTINATION')

jobsmap

jobs_heatmap.save("./jobs_heatmap_by_month_destination.html")

In [5]:
# mon = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

# for m in range(12):  # cycle through months

#     mapJobs(dfJobs[pd.DatetimeIndex(dfJobs['DTSTART']).month == (m+1)], 'jobMap{}.html'.format(mon[m]))